In [19]:
!pip install txtai[all] > /dev/null

In [24]:
from txtai.workflow import Task
from txtai.pipeline import Summary

t1 = Task(lambda x: [y * 2 for y in x])

t2 = Task(Summary())

t3 = Task(lambda x: [y - 2 for y in x])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [23]:
from txtai.workflow import Workflow

workT2 = Workflow([t2])

list(workT2(["This is a super long text. Summarize it to 1 word"]))

['This is a super long text. Summarize it to 1 word']

In [26]:
wrk_chn1 = Workflow([t1, t3])

In [ ]:
#the input moves through each of the tasks and reaches output

In [28]:
list(wrk_chn1([2,8]))

[2, 14]

### Multi-action task merges

Multi-action tasks will generate parallel outputs for the input data. The task output can be merged together in a couple different ways.

Each function / pipeline is individual action

In [36]:
#There are different ways of initiating tasks

t1_u = Task([lambda x: [y * 2 for y in x],
             lambda x: [y - 2 for y in x]],
            unpack=True, column=0)

In [37]:
wrk_chn2 = Workflow([t1_u])

In [38]:
list(wrk_chn2([2,8]))

[(4, 0), (16, 6)]

In [46]:
t2_u = Task([lambda x: [y * 2 for y in x],
             lambda x: [y - 2 for y in x]],
            unpack=False, column={0:0,1:1})

wrk_chn3 = Workflow([t2_u])

list(wrk_chn3([2,8,7]))

[(4, 0), (16, 6), (14, 5)]

In [50]:
t3_u = Task([lambda x: [y * 2 for y in x],
             lambda x: [y - 2 for y in x]],
            unpack=False, 
            column=0, #no change if {0:0,1:1} used
            merge='hstack')

wrk_chn4 = Workflow([t3_u])

list(wrk_chn4([2,8,7]))

[(4, 0), (16, 6), (14, 5)]

In [51]:
t4_u = Task([lambda x: [y * 2 for y in x],
             lambda x: [y - 2 for y in x]],
            unpack=False, 
            column=0, #no change if {0:0,1:1} used
            merge='vstack')

wrk_chn5 = Workflow([t4_u])

list(wrk_chn5([2,8,7]))

[4, 0, 16, 6, 14, 5]

In [52]:
t5_u = Task([lambda x: [y * 2 for y in x],
             lambda x: [y - 2 for y in x]],
            unpack=False, 
            column=0, #no change if {0:0,1:1} used
            merge='concat')

wrk_chn6 = Workflow([t5_u])

list(wrk_chn6([2,8,7]))

['4', '16. 6', '14. 5']

In [60]:
from txtai.workflow import FileTask, Workflow, ConsoleTask

#the work flow works in the expected way of input and output
tlen = Task(lambda x: [len(y) for y in x])

workflow = Workflow([tlen, t5_u, ConsoleTask()])

In [61]:
list(workflow(["there_five","your_two"]))

Inputs: [
  "20. 8",
  "16. 6"
]
Outputs: [
  "20. 8",
  "16. 6"
]


['20. 8', '16. 6']

In [111]:
from txtai.workflow import ExportTask

expFlow = Workflow([ExportTask(tlen)]) # Unsure how it works

In [ ]:
list(expFlow(['Tere bin','your self']))

In [70]:
workflow = Workflow([FileTask()])

list(workflow(["/content/tmux_conf_local.txt", "/content/linux_play.txt"]))

['/content/tmux_conf_local.txt', '/content/linux_play.txt']

In [75]:
data_read = Task(lambda x: [open(y).read() for y in x])
data_write = Task(lambda x: [open(y).write() for y in x])

workflow = Workflow([FileTask(), 
                     data_read, 
                     ConsoleTask()])

In [ ]:
#File task & console output tasks makes sense
list(workflow(["/content/tmux_conf_local.txt", 
               "/content/linux_play.txt"]))

In [76]:
#write flow is not as expected
writeflow = Workflow([Task(lambda x: [y * 2 for y in x]),
                      FileTask(), 
                     data_write])

In [ ]:
list(writeflow([5,7]))

In [ ]:
# Get test data
!wget -N https://github.com/neuml/txtai/releases/download/v3.5.0/tests.tar.gz
!tar -xvzf tests.tar.gz

In [84]:
from txtai.workflow import ImageTask, Workflow
from txtai.pipeline import Caption

get_caption = Caption()

captionTask = Task(lambda z: get_caption(z))

workflow = Workflow([ImageTask(),captionTask,ConsoleTask()])

No model was supplied, defaulted to ydshieh/vit-gpt2-coco-en and revision 65636df (https://huggingface.co/ydshieh/vit-gpt2-coco-en).
Using a pipeline without specifying a model name and revision in production is not recommended.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
import glob

list(workflow(glob.glob("/content/txtai/*.jpg")))

In [89]:
from txtai.workflow import RetrieveTask, Workflow

workflow = Workflow([RetrieveTask(directory="/content/")])

In [90]:
list(workflow(["https://github.com/neuml/txtai/releases/download/v3.5.0/tests.tar.gz"]))

['/content/tests.tar.gz']

In [96]:
from txtai.workflow import ServiceTask, Workflow

workflow = Workflow([ServiceTask(url="https://pyrite-ethereal-soccer.glitch.me/user/v1/show_users",
                                 method="GET")])

In [ ]:
list(workflow(["parameter"]))

In [102]:
from txtai.workflow import TemplateTask, Workflow

workflow = Workflow([TemplateTask(template="This is a {text} task")])

list(workflow([{"text": "newer"}]))

['This is a newer task']

We can discuss on following workflows

1) Console

2) File : for reading

3) Image

4) Retrieve 

5) Template

6) Workflow